## Что нужно сделать?
    - переделать получение описаний с запроса к api на поиск по данной html-странице

### Входные данные
    > на входе мы получаем несколько тысяч вакансий, краткую информацию о них (дата публикации, город, описание)
    

In [1]:
import requests
import json
import sqlite3
import re
from bs4 import BeautifulSoup
import pymorphy2
import string
import collections
import itertools
import nltk

In [2]:
dbs = [
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\sberbank_hh.db',
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\drweb_hh.db',
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\ifmo_hh.db',
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\kasper_hh.db',
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\ozon_job.db',
    'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\tinkoff_hh.db'
]

In [3]:
original_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\sberbank_hh.db'

conn = sqlite3.connect(original_db)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
original = cursor.fetchall()
conn.close()

In [4]:
filtered_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\filtered.db'

conn = sqlite3.connect(filtered_db)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancies')
filtered = cursor.fetchall()
conn.close()

In [2]:
new_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\drweb_hh.db'

conn = sqlite3.connect(new_db)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
new_base = cursor.fetchall()
conn.close()

In [180]:
filtered[1]

('https://hh.ru/vacancy/29654641',
 'Frontend разработчик',
 'Москва',
 '2019-01-15',
 'Требования:   Опыт web разработки от 3 лет (HTML, HTML5, CSS, JavaScript, препроцессинг CSS) Опыт разработки на библиотеках и фреймворках (AngularJS, ReactJS) Активное использование инструментов контроля версий (Git, CVS) Умение работать с базами данных и языками запросов (SQL) Опыт работы с серверными технологиями (Node.js) Взаимодействие с бекендом (SOAP/REST)    Условия работы: Мы предлагаем:    трудоустройство согласно ТК РФ; гарантированный доход плюс премиальное вознаграждение; возможность работать рядом с домом/местом учебы; регулярное корпоративное обучение; ДМС, страхование от несчастных случаев и тяжелых заболеваний; материальную помощь и социальную поддержку, корпоративную пенсионную программу; льготные условия кредитования; яркую и насыщенную корпоративную жизнь.  Мы ждем тебя, чтобы строить счастливое будущее вместе!')

#### Первичная обработка вакансий (выделение IT-вакансий) 

In [9]:
def get_descr(raw_html):
    soup = BeautifulSoup(raw_html)
    return soup.find('div', class_='g-user-content')

In [10]:
without_dupl = []
ids = []
for job in new_base:
    id_ = job[1].split('/')[-1]
    if id_ not in ids:
        ids.append(id_)
        without_dupl.append({'id': id_, 'description': get_descr(job[5])})

In [17]:
len(without_dupl)

206

In [16]:
filtered_new = []
for vacancy in without_dupl:
    id_ = vacancy['id'].split('/')[-1]
    req = requests.get('https://api.hh.ru/vacancies/' + id_).json()
    try:
        profarea = req['specializations'][0]['profarea_id']
        if profarea == '1':
            filtered_new.append(vacancy)
    except:
        pass

In [18]:
len(filtered_new)

134

In [76]:
descrs = {}
left = 369
for job in filtered:
    id_ = job[0].split('/')[-1]
    desc = requests.get('https://api.hh.ru/vacancies/' + id_).json()['description']
    descrs[id_] = desc
    left -= 1
    print(f'осталось обработать {round(left/369 * 100, 1)} % вакансий.')

осталось обработать 99.7 % вакансий.
осталось обработать 99.5 % вакансий.
осталось обработать 99.2 % вакансий.
осталось обработать 98.9 % вакансий.
осталось обработать 98.6 % вакансий.
осталось обработать 98.4 % вакансий.
осталось обработать 98.1 % вакансий.
осталось обработать 97.8 % вакансий.
осталось обработать 97.6 % вакансий.
осталось обработать 97.3 % вакансий.
осталось обработать 97.0 % вакансий.
осталось обработать 96.7 % вакансий.
осталось обработать 96.5 % вакансий.
осталось обработать 96.2 % вакансий.
осталось обработать 95.9 % вакансий.
осталось обработать 95.7 % вакансий.
осталось обработать 95.4 % вакансий.
осталось обработать 95.1 % вакансий.
осталось обработать 94.9 % вакансий.
осталось обработать 94.6 % вакансий.
осталось обработать 94.3 % вакансий.
осталось обработать 94.0 % вакансий.
осталось обработать 93.8 % вакансий.
осталось обработать 93.5 % вакансий.
осталось обработать 93.2 % вакансий.
осталось обработать 93.0 % вакансий.
осталось обработать 92.7 % вакансий.
о

осталось обработать 39.3 % вакансий.
осталось обработать 39.0 % вакансий.
осталось обработать 38.8 % вакансий.
осталось обработать 38.5 % вакансий.
осталось обработать 38.2 % вакансий.
осталось обработать 37.9 % вакансий.
осталось обработать 37.7 % вакансий.
осталось обработать 37.4 % вакансий.
осталось обработать 37.1 % вакансий.
осталось обработать 36.9 % вакансий.
осталось обработать 36.6 % вакансий.
осталось обработать 36.3 % вакансий.
осталось обработать 36.0 % вакансий.
осталось обработать 35.8 % вакансий.
осталось обработать 35.5 % вакансий.
осталось обработать 35.2 % вакансий.
осталось обработать 35.0 % вакансий.
осталось обработать 34.7 % вакансий.
осталось обработать 34.4 % вакансий.
осталось обработать 34.1 % вакансий.
осталось обработать 33.9 % вакансий.
осталось обработать 33.6 % вакансий.
осталось обработать 33.3 % вакансий.
осталось обработать 33.1 % вакансий.
осталось обработать 32.8 % вакансий.
осталось обработать 32.5 % вакансий.
осталось обработать 32.2 % вакансий.
о

#### Сбор всех заголовков, выделение нужных (вручную!!!1) + анализ

In [78]:
punct = '«»' + string.punctuation

In [79]:
headers = []
table = str.maketrans({key: None for key in punct})
for desc in descrs:
    new_headers = [d for d in [x.text.translate(table).strip().lower() for x in BeautifulSoup(descrs[desc]).findAll('strong')] 
                       if 1 < len(d) < 30]
    headers = headers + new_headers

In [80]:
for head in headers:
    if set('abcdefghijklmnopqrstuvwxyz') & set(head) != set():
        headers.remove(head)

In [81]:
print(len(headers))      # неуникальных
print(len(set(headers))) # уникальных

1150
247


In [82]:
headers[:7]

['обязанности',
 'требования',
 'условия',
 'требования',
 'мы предлагаем',
 'требования',
 'мы предлагаем']

In [83]:
counter_d = sorted(collections.Counter(headers).items(), key = lambda x: x[1], reverse=True)
counter_d[:6]

[('требования', 178),
 ('условия', 144),
 ('обязанности', 136),
 ('мы предлагаем', 42),
 ('задачи', 39),
 ('условия работы', 31)]

In [84]:
keywords = ['требования', 'требуется', 'если', 'ждем', 'ждём', 'ожидаем', 'ищем', 'будет', 'компетенции', 'плюсом', 'желательно'
           'знание', 'стек', 'стэк', 'технологии', 'навыки', 'обязательно', 'от вас', 'опыт', 'ожидания']

In [85]:
def check(word_list):
    keywords = ['требования', 'требуется', 'если', 'ждем', 'ждём', 'ожидаем', 'ищем', 
                'будет', 'компетенции', 'плюсом', 'желательно', 'знание', 'стек', 
                'стэк', 'технологии', 'навыки', 'обязательно', 'от вас', 'опыт', 'ожидания']
    
    return set(keywords) & set(word_list) != set()

In [86]:
for head in set(headers):
    word_list = head.split()
    if set(keywords) & set(word_list) != set():
        print(head)

компетенции
кого ждем
кого мы ищем
ожидания от кандидата
круто если вы так же
будет преимуществом
что мы от вас ожидаем
дополнительные требования
будет плюсом
приветствуются знания  опыт
мы ожидаем что вы
профессиональные навыки
мы ожидаем от кандидата
если есть
что нужно будет делать
инструментальные требования
требования к кандидатам
стэк
дополнительным плюсом будет
плюсом будет
круто если вы
мы ищем
обязательные компетенции
мы ждем что вы
плюсом будут
ты подходишь нам если ты
знания и навыки
ы ждем тебя
кого мы ждём
технические компетенции
необходимый опыт
навыки
необходимые навыки
ждем
опыт управления проектами
мы ожидаем
мы ждем тебя в нашу команду
ты подходишь нам если
ожидаем от кандидата
мы ждём от тебя
обязательные навыки
от вас мы ждем
плюсом будет опыт работы с
чего ждем от кандидата
мы ожидаем от вас
будет здорово если вы
вашим преимуществом будет
если ты
мы ожидаем что ты
обязательные требования
если тебе интересно
опыт
ждем от вас
обязательно
основные требования
дополните

#### Выделение нужного абзаца

In [153]:
#errors = []
len(errors)

90

In [88]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return ' '.join(cleantext.split())

In [89]:
punct = '«»' + string.punctuation
table = str.maketrans({key: None for key in punct})

In [140]:
all_paragraphs = {}
for id_ in descrs:
    text = descrs[id_]
    if '<strong>' in text:
        paragraphs = [x.split('</strong>') for x in text.split('<strong>') if '</strong>' in x]
        all_paragraphs[id_] = {x[0].translate(table).strip().lower(): x[1] for x in paragraphs}
    else:
        errors.append(id_)

In [141]:
all_paragraphs['29654641']

{'требования': '</p><p>Опыт web разработки от 3 лет (HTML, HTML5, CSS, JavaScript, препроцессинг CSS)<br />Опыт разработки на библиотеках и фреймворках (AngularJS, ReactJS)<br />Активное использование инструментов контроля версий (Git, CVS)<br />Умение работать с базами данных и языками запросов (SQL)<br />Опыт работы с серверными технологиями (Node.js)<br />Взаимодействие с бекендом (SOAP/REST)</p><div><p></p><p><b>Условия работы:</b></p><p>',
 'мы предлагаем': '</p><p> </p><ul><li>трудоустройство согласно ТК РФ;</li><li>гарантированный доход плюс премиальное вознаграждение;</li><li>возможность работать рядом с домом/местом учебы;</li><li>регулярное корпоративное обучение;</li><li>ДМС, страхование от несчастных случаев и тяжелых заболеваний;</li><li>материальную помощь и социальную поддержку, корпоративную пенсионную программу;</li><li>льготные условия кредитования;</li><li>яркую и насыщенную корпоративную жизнь.<br /></li></ul><p>',
 'мы ждем тебя чтобы строить счастливое будущее вме

In [142]:
requirements = {}
for job in all_paragraphs: 
    for paragraph in all_paragraphs[job]:
        word_list = paragraph.split()
        if set(word_list) & set(keywords) != set():
            new_reqs = cleanhtml(all_paragraphs[job][paragraph])
            if new_reqs != '':
                requirements[job] = cleanhtml(all_paragraphs[job][paragraph])

In [152]:
errors = errors + list(set(all_paragraphs.keys()).difference(set(requirements.keys()))) # добавил потерянные вакансии

In [155]:
requirements['29654641']

'Опыт web разработки от 3 лет (HTML, HTML5, CSS, JavaScript, препроцессинг CSS) Опыт разработки на библиотеках и фреймворках (AngularJS, ReactJS) Активное использование инструментов контроля версий (Git, CVS) Умение работать с базами данных и языками запросов (SQL) Опыт работы с серверными технологиями (Node.js) Взаимодействие с бекендом (SOAP/REST) Условия работы:'